# Market Microstructure Observatory

This notebook operationalises the Market Microstructure Observatory called
out in the High-Impact Development Roadmap. It demonstrates how the new
tiered archival process can be analysed to surface liquidity regimes and
anomaly clusters.

In [ ]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Dict, List

import pandas as pd

RAW_PATH = Path('docs/microstructure_raw_data.json')
records: List[Dict[str, object]] = []
if RAW_PATH.exists():
    with RAW_PATH.open('r', encoding='utf-8') as handle:
        for line in handle:
            line = line.strip()
            if not line:
                continue
            records.append(json.loads(line))

df = pd.DataFrame.from_records(records)
df.head()

In [ ]:
if df.empty:
    print('Dataset is empty – run scripts/archive_microstructure_datasets.py to hydrate inputs.')
else:
    volume_profile = (
        df.groupby('symbol')['executed_volume'].sum().sort_values(ascending=False)
    )
    display(volume_profile.to_frame(name='executed_volume'))

    imbalance = df['bid_volume'] - df['ask_volume']
    df.assign(orderbook_imbalance=imbalance).plot(
        x='timestamp', y='orderbook_imbalance', title='Order Book Imbalance Trend', figsize=(10, 4)
    )